# Drought Prediction

## Load Libraries

In [ ]:
# General purpose libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import pickle
import math

# Scikit-learn libraries for machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, ParameterGrid
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA, KernelPCA
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree, metrics
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay, classification_report,
                             accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, roc_curve, auc, cohen_kappa_score)
from sklearn.naive_bayes import GaussianNB

# Imbalanced-learn libraries for handling imbalanced datasets
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NeighbourhoodCleaningRule, NearMiss

# PyTorch libraries for deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split
from torchviz import make_dot
import random
from torch.utils.tensorboard import SummaryWriter

# Scipy library for statistical functions
from scipy.stats import uniform
# Base classes for custom estimators in scikit-learn
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
# Check device, use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## Data Load and Setup

In [ ]:
# drought_df =  pd.read_csv('data/all_timeseries.csv')

# Load training and testing data from a pickle file
with open('data/Xy_trainTest.pkl', 'rb') as f:
    # Unpickle the data into training and testing datasets
    X_train, X_test, y_train, y_test = pickle.load(f)

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [ ]:
# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [ ]:
# Define the validation split ratio
val_split_ratio = 0.2
val_size = int(len(train_dataset) * val_split_ratio)
train_size = len(train_dataset) - val_size

# Split the dataset
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=4, pin_memory=True)

## Define Neural Network Classes/Functions

#### Class: DroughtClassifier

In [ ]:
class DroughtClassifier(nn.Module):
    """
    A neural network classifier for drought prediction.

    Args:
        input_size (int): The number of input features.
        hidden_sizes (list of int): A list containing the sizes of the hidden layers.
        output_size (int): The number of output classes.
        dropout_prob (float, optional): The probability of an element to be zeroed in dropout. Default is 0.5.

    Attributes:
        layers (nn.ModuleList): A list of linear layers.
        activations (nn.ModuleList): A list of activation functions.
        dropout (nn.Dropout): Dropout layer for regularization.
    """
    def __init__(self, input_size, hidden_sizes, output_size, dropout_prob=0.5):
        super(DroughtClassifier, self).__init__()
        self.layers = nn.ModuleList()
        self.activations = nn.ModuleList()
        
        # Input layer
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        self.activations.append(nn.ReLU())
        
        # Hidden layers
        for i in range(len(hidden_sizes) - 1):
            self.layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            self.activations.append(nn.ReLU())
        
        # Output layer
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        """
        Defines the forward pass of the neural network.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output tensor after passing through the network.
        """
        for layer, activation in zip(self.layers[:-1], self.activations):
            x = self.dropout(activation(layer(x)))
        x = self.layers[-1](x)
        return x


#### Class: EarlyStopping

In [ ]:
class EarlyStopping:
    """
    Early stopping to stop the training when the loss does not improve after a given patience.

    Args:
        patience (int, optional): How long to wait after last time validation loss improved. Default is 5.
        delta (float, optional): Minimum change in the monitored quantity to qualify as an improvement. Default is 0.

    Attributes:
        patience (int): How long to wait after last time validation loss improved.
        delta (float): Minimum change in the monitored quantity to qualify as an improvement.
        best_loss (float): Best recorded validation loss.
        counter (int): Counter for how many epochs have passed since the last improvement.
        early_stop (bool): Whether early stopping is triggered.
    """
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        """
        Checks if the validation loss has improved and updates the counter and early stop flag accordingly.

        Args:
            val_loss (float): The current validation loss.

        Returns:
            None
        """
        if val_loss < self.best_loss - self.delta:
            # If the validation loss has improved (by more than delta), reset the counter
            self.best_loss = val_loss
            self.counter = 0
        else:
            # If the validation loss has not improved, increment the counter
            self.counter += 1
            if self.counter >= self.patience:
                # If the counter exceeds the patience, set the early stop flag
                self.early_stop = True


#### Function: get_log_dir

In [ ]:
# Define a function to get a unique log directory
def get_log_dir(base_dir='runs'):
    """
    Generates a unique log directory path based on the current date and time.

    Args:
        base_dir (str, optional): The base directory where logs will be saved. Default is 'runs'.

    Returns:
        str: A unique directory path for saving logs.
    """
    # Get the current date and time as a formatted string
    current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    # Create a log directory path by combining the base directory and the current time
    log_dir = os.path.join(base_dir, current_time)
    return log_dir

#### Function train_model

In [ ]:
# Training function with early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25, patience=5, log_dir=None, hparams=None):
    """
    Trains the model with early stopping and logs metrics to TensorBoard.

    Args:
        model (nn.Module): The neural network model to be trained.
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        criterion (nn.Module): The loss function.
        optimizer (torch.optim.Optimizer): The optimizer for training.
        scheduler (torch.optim.lr_scheduler): Learning rate scheduler.
        num_epochs (int, optional): The maximum number of epochs for training. Default is 25.
        patience (int, optional): The number of epochs with no improvement after which training will be stopped. Default is 5.
        log_dir (str, optional): The directory to save TensorBoard logs. If None, a new directory will be created.
        hparams (dict, optional): Hyperparameters to log.

    Returns:
        None
    """
    if log_dir is None:
        log_dir = get_log_dir()                         # Create a unique log directory if not provided
    writer = SummaryWriter(log_dir=log_dir)             # Initialize TensorBoard writer
    early_stopping = EarlyStopping(patience=patience)   # Initialize early stopping
    
    # Log hyperparameters before training begins
    if hparams is not None:
        writer.add_hparams(hparams, {})

    # For each epoch
    for epoch in range(num_epochs):
        model.train()                # Set model to training mode
        running_loss = 0.0
        correct_predictions = 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
            
            optimizer.zero_grad() # Zero the parameter gradients
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)             # Accumulate loss
            _, preds = torch.max(outputs, 1)                         # Get predictions
            correct_predictions += torch.sum(preds == labels).item() # Count correct predictions
        
        epoch_loss = running_loss / len(train_loader.dataset)             # Calculate average loss for this epoch
        epoch_accuracy = correct_predictions / len(train_loader.dataset)  # Calculate accuracy for this epoch
        
        # Validation loop
        model.eval() # Set model to evaluation mode
        val_running_loss = 0.0
        val_correct_predictions = 0
        with torch.no_grad():                                           # Disable gradient computation for validation
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)   # Move inputs and labels to GPU
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)        # Accumulate validation loss
                _, preds = torch.max(outputs, 1)                        # Get predictions
                val_correct_predictions += torch.sum(preds == labels).item() # Count correct predictions
        
        val_loss = val_running_loss / len(val_loader.dataset)               # Calculate average validation loss
        val_accuracy = val_correct_predictions / len(val_loader.dataset)    # Calculate validation accuracy
        
        # Log metrics to TensorBoard
        writer.add_scalar('Loss/train', epoch_loss, epoch)                      # Log training loss
        writer.add_scalar('Loss/validation', val_loss, epoch)                   # Log validation loss
        writer.add_scalar('Accuracy/train', epoch_accuracy, epoch)              # Log training accuracy
        writer.add_scalar('Accuracy/validation', val_accuracy, epoch)           # Log validation accuracy
        writer.add_scalar('Learning_Rate', scheduler.get_last_lr()[0], epoch)   # Log learning rate
        
        # Print metrics for the current epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        # Update learning rate
        scheduler.step()

        # Check early stopping criteria
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Log final metrics to TensorBoard with hyperparameters
    if hparams is not None:
        writer.add_hparams(hparams, {'hparam/accuracy': val_accuracy, 'hparam/loss': val_loss})
    print('Training complete')
    writer.close()

#### Function evaluate_model

In [ ]:
# Evaluation function
def evaluate_model(model, test_loader, criterion):
    """
    Evaluates the model on the test dataset and prints the test loss and accuracy.

    Args:
        model (nn.Module): The trained neural network model to be evaluated.
        test_loader (DataLoader): DataLoader for the test dataset.
        criterion (nn.Module): The loss function.

    Returns:
        None
    """
    model.eval() # Set model to evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    
    with torch.no_grad():                                           # Disable gradient computation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)   # Move inputs and labels to GPU

            outputs = model(inputs)             # Forward pass
            loss = criterion(outputs, labels)   # Compute loss

            running_loss += loss.item() * inputs.size(0)    # Accumulate loss
            
            _, preds = torch.max(outputs, 1)                            # Get predictions
            correct_predictions += torch.sum(preds == labels).item()    # Count correct predictions
    
    test_loss = running_loss / len(test_loader.dataset)         # Calculate average test loss
    accuracy = correct_predictions / len(test_loader.dataset)   # Calculate test accuracy
    
    # Print test metrics
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

#### Class: PyTorchClassifier

In [ ]:
# Define a custom PyTorch classifier for hyperparameter search
class PyTorchClassifier(BaseEstimator, ClassifierMixin):
    """
    Custom PyTorch classifier for hyperparameter search with scikit-learn compatibility.

    Args:
        hidden_sizes (tuple): Sizes of hidden layers.
        dropout_prob (float): Dropout probability.
        lr (float): Learning rate.
        num_epochs (int): Number of epochs to train.
        patience (int): Patience for early stopping.
        log_dir (str): Directory to save TensorBoard logs.
    """
    def __init__(self, hidden_sizes=(512, 128, 64, 32), dropout_prob=0.5, lr=0.001, num_epochs=3, patience=5, log_dir=None):
        self.hidden_sizes = hidden_sizes
        self.dropout_prob = dropout_prob
        self.lr = lr
        self.num_epochs = num_epochs
        self.patience = patience
        self.log_dir = log_dir
        self.model = None

    def fit(self, X, y):
        """
        Train the PyTorch model on the given dataset.

        Args:
            X (numpy.ndarray): Training data features.
            y (numpy.ndarray): Training data labels.

        Returns:
            self: Returns an instance of self.
        """
        input_size = X.shape[1]         # Number of input features
        output_size = len(np.unique(y)) # Number of unique classes

        # Initialize the model
        self.model = DroughtClassifier(input_size, self.hidden_sizes, output_size, self.dropout_prob).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.005)

        # Create DataLoader for training data
        train_tensor = torch.tensor(X, dtype=torch.float32)
        labels_tensor = torch.tensor(y, dtype=torch.long)
        train_dataset = TensorDataset(train_tensor, labels_tensor)
        train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=4, pin_memory=True)
        
        # Define hyperparameters for logging
        hparams = {
            'hidden_sizes': str(self.hidden_sizes),
            'dropout_prob': self.dropout_prob,
            'lr': self.lr
        }

        # Train the model
        train_model(self.model, train_loader, val_loader, criterion, optimizer, scheduler, self.num_epochs, self.patience, log_dir=self.log_dir, hparams=hparams)
        return self

    def predict(self, X):
        """
        Predict the labels for the given dataset.

        Args:
            X (numpy.ndarray): Data features.

        Returns:
            numpy.ndarray: Predicted labels.
        """
        self.model.eval()   # Set model to evaluation mode
        test_tensor = torch.tensor(X, dtype=torch.float32)
        test_loader = DataLoader(test_tensor, batch_size=512, shuffle=False, num_workers=4, pin_memory=True)
        predictions = []

        with torch.no_grad():  # Disable gradient computation
            for inputs in test_loader:
                inputs = inputs.to(device)              # Move inputs to GPU
                outputs = self.model(inputs)            # Forward pass
                _, preds = torch.max(outputs, 1)        # Get predictions
                predictions.extend(preds.cpu().numpy()) # Store predictions

        # Return predictions as a numpy array
        return np.array(predictions)

## Initialize the Base Model, Loss Function, and Optimizer

In [ ]:
# Define model parameters
input_size = X_train.shape[1]       # Number of features
hidden_sizes = [512, 128, 64, 32]   

output_size = 6                # Number of output classes
dropout_prob = 0.5             # Dropout probability

# Initialize the model
base_model = DroughtClassifier(input_size, hidden_sizes, output_size, dropout_prob).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(base_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.005)
# writer = SummaryWriter()

## ParameterGrid Search

In [ ]:
# Define hyperparameter grid for RandomizedSearchCV
lr_space = [10**(-4 * np.random.uniform(0.5, 1)) for _ in range(5)] #learning rate values between 0.1 (1e-1) and 0.001 (1e-4)
param_grid = {
    'hidden_sizes': [(512, 128, 64, 32), (256, 64, 32), (512, 256, 128, 64)],
    'dropout_prob': [0.3, 0.4, 0.5],
    'lr': lr_space
}

# Initialize ParameterGrid
grid = ParameterGrid(param_grid)

In [ ]:
# Ensure y_test is a numpy array
y_test = y_test.to_numpy() if not isinstance(y_test, np.ndarray) else y_test

In [ ]:
# Iterate over each hyperparameter combination
best_model = None
best_params = None
best_val_accuracy = 0

for params in grid:
    # Print the current hyperparameter combination being trained
    print(f"Training with parameters: {params}")

    # Generate a unique log directory for each iteration
    log_dir = get_log_dir()

    # Create a PyTorchClassifier model with the current hyperparameters
    model = PyTorchClassifier(hidden_sizes=params['hidden_sizes'],dropout_prob=params['dropout_prob'],lr=params['lr'],num_epochs=25,patience=3,log_dir=log_dir)
    
    # Train the model on the training data
    model.fit(X_train, y_train)

    # Evaluate on validation set
    val_predictions = model.predict(X_test)
    val_accuracy = accuracy_score(y_test, val_predictions)

    # Print the validation accuracy for the current hyperparameters
    print(f"Validation Accuracy: {val_accuracy:.4f}")

    # Update the best model and parameters if the current model performs better
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = model
        best_params = params

In [ ]:
# Save the best model and parameters
with open('saved_models/parameterGridSearch_bestModel.pkl', 'wb') as f:
    pickle.dump(best_model, f)

with open('saved_models/parameterGridSearch_bestParams.pkl', 'wb') as f:
    pickle.dump(best_params, f)

print(f"Best Hyperparameters: {best_params}")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")

# Evaluate the best model using the test set
evaluate_model(best_model.model, test_loader, criterion)

In [ ]:
# Load the best model and parameters
with open('saved_models/best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

with open('saved_models/best_params.pkl', 'rb') as f:
    best_params = pickle.load(f)

print(f"Loaded Best Hyperparameters: {best_params}")

## Learning Rate Scheduler

In [ ]:
# Placeholder variables for the best scheduler parameters
best_step_lr_params = None
best_step_lr_val_accuracy = 0

best_cosine_annealing_lr_params = None
best_cosine_annealing_lr_val_accuracy = 0

best_cosine_annealing_lr_warm_params = None
best_cosine_annealing_lr_warm_val_accuracy = 0

# Define the grid of StepLR parameters to search
step_lr_grid = [
    {'scheduler_type': 'StepLR', 'step_size': 10, 'gamma': 0.25},
    {'scheduler_type': 'StepLR', 'step_size': 10, 'gamma': 0.5},
    {'scheduler_type': 'StepLR', 'step_size': 20, 'gamma': 0.25},
    {'scheduler_type': 'StepLR', 'step_size': 20, 'gamma': 0.5},
]

# Define the grid of CosineAnnealingLR parameters to search
cosine_annealing_lr_grid = [
    {'scheduler_type': 'CosineAnnealingLR', 'T_max': 10, 'eta_min': 1e-5},
    {'scheduler_type': 'CosineAnnealingLR', 'T_max': 20, 'eta_min': 1e-5},
    {'scheduler_type': 'CosineAnnealingLR', 'T_max': 10, 'eta_min': 1e-6},
    {'scheduler_type': 'CosineAnnealingLR', 'T_max': 20, 'eta_min': 1e-6},
]
cosine_annealing_warm_lr_grid = [
    {'scheduler_type': 'CosineAnnealingLRWarm', 'T_0': 10, 'eta_min': 1e-4},
    {'scheduler_type': 'CosineAnnealingLRWarm', 'T_0': 20, 'eta_min': 1e-4},
    {'scheduler_type': 'CosineAnnealingLRWarm', 'T_0': 10, 'eta_min': 1e-5},
    {'scheduler_type': 'CosineAnnealingLRWarm', 'T_0': 20, 'eta_min': 1e-5},
]

In [ ]:
# Function to create the scheduler based on parameters
def create_scheduler(optimizer, params):
    if params['scheduler_type'] == 'StepLR':
        return StepLR(optimizer, step_size=params['step_size'], gamma=params['gamma'])
    elif params['scheduler_type'] == 'CosineAnnealingLR':
        return CosineAnnealingLR(optimizer, T_max=params['T_max'], eta_min=params['eta_min'])
    elif params['scheduler_type'] == 'CosineAnnealingLRWarm':
        return CosineAnnealingWarmRestarts(optimizer, 
                                        T_0 = params['T_0'],            # Number of iterations for the first restart
                                        T_mult = 1,                     # A factor increases TiTi​ after a restart
                                        eta_min = params['eta_min'])    # Minimum learning rate
    else:
        raise ValueError(f"Unknown scheduler type: {params['scheduler_type']}")

#### StepLR

In [ ]:
# Run for StepLR parameters
for scheduler_params in step_lr_grid:
    print(f"Tuning with StepLR parameters: {scheduler_params}")
    
    # Initialize a new model for each scheduler parameter combination
    model = DroughtClassifier(input_size, best_params['hidden_sizes'], output_size, best_params['dropout_prob']).to(device)
    
    # Create a new optimizer and scheduler with the current scheduler parameters
    optimizer = optim.Adam(model.parameters(), lr=best_params['lr'])
    scheduler = create_scheduler(optimizer, scheduler_params)

    # Train the model with the new scheduler parameters
    train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=5, log_dir=get_log_dir(), hparams=scheduler_params)
    
    # Evaluate on the validation set
    model.eval()
    val_correct_preds = 0
    val_total_preds = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct_preds += (predicted == labels).sum().item()
            val_total_preds += labels.size(0)
    
    val_accuracy = val_correct_preds / val_total_preds
    print(f"Validation Accuracy with StepLR parameters: {val_accuracy:.4f}")
    
    # Update the best StepLR parameters if the current model performs better
    if val_accuracy > best_step_lr_val_accuracy:
        best_step_lr_val_accuracy = val_accuracy
        best_step_lr_params = scheduler_params

print(f"Best StepLR parameters: {best_step_lr_params}")
print(f"Best StepLR validation accuracy: {best_step_lr_val_accuracy:.4f}")

In [ ]:
# Save the best model and parameters
with open('saved_models/stepLRSearch_bestParams.pkl', 'wb') as f:
    pickle.dump(best_step_lr_params, f)

In [ ]:
# Load the saved best model
with open('saved_models/stepLRSearch_bestParams.pkl', 'rb') as f:
    best_step_lr_params = pickle.load(f)

print(f"Loaded Best Hyperparameters: {best_step_lr_params}")

#### CosineAnnealingLR

In [ ]:
# Run for CosineAnnealingLR parameters
for scheduler_params in cosine_annealing_lr_grid:
    print(f"Tuning with CosineAnnealingLR parameters: {scheduler_params}")
    
    # Initialize a new model for each scheduler parameter combination
    model = DroughtClassifier(input_size, best_params['hidden_sizes'], output_size, best_params['dropout_prob']).to(device)
    
    # Create a new optimizer and scheduler with the current scheduler parameters
    optimizer = optim.Adam(model.parameters(), lr=best_params['lr'])
    scheduler = create_scheduler(optimizer, scheduler_params)

    # Train the model with the new scheduler parameters
    train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25, patience=5, log_dir=get_log_dir(), hparams=scheduler_params)
    
    # Evaluate on the validation set
    model.eval()
    val_correct_preds = 0
    val_total_preds = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct_preds += (predicted == labels).sum().item()
            val_total_preds += labels.size(0)
    
    val_accuracy = val_correct_preds / val_total_preds
    print(f"Validation Accuracy with CosineAnnealingLR parameters: {val_accuracy:.4f}")
    
    # Update the best CosineAnnealingLR parameters if the current model performs better
    if val_accuracy > best_cosine_annealing_lr_val_accuracy:
        best_cosine_annealing_lr_val_accuracy = val_accuracy
        best_cosine_annealing_lr_params = scheduler_params

print(f"Best CosineAnnealingLR parameters: {best_cosine_annealing_lr_params}")
print(f"Best CosineAnnealingLR validation accuracy: {best_cosine_annealing_lr_val_accuracy:.4f}")


In [ ]:
# Save the best model
with open('saved_models/CosineAnnealingLR_bestParams.pkl', 'wb') as f:
    pickle.dump(best_cosine_annealing_lr_params, f)

In [ ]:
# Load the best model
with open('saved_models/CosineAnnealingLR_bestParams.pkl', 'rb') as f:
    best_cosine_annealing_lr_params = pickle.load(f)

print(f"Loaded Best Hyperparameters: {best_cosine_annealing_lr_params}")

#### CosineAnnealingLR_Warm

In [ ]:
# Run for CosineAnnealingLR_warm parameters
for scheduler_params in cosine_annealing_warm_lr_grid:
    print(f"Tuning with CosineAnnealingLR_warm parameters: {scheduler_params}")
    
    # Initialize a new model for each scheduler parameter combination
    model = DroughtClassifier(input_size, best_params['hidden_sizes'], output_size, best_params['dropout_prob']).to(device)
    
    # Create a new optimizer and scheduler with the current scheduler parameters
    optimizer = optim.Adam(model.parameters(), lr=best_params['lr'])
    scheduler = create_scheduler(optimizer, scheduler_params)

    # Train the model with the new scheduler parameters
    train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=5, log_dir=get_log_dir(), hparams=scheduler_params)
    
    # Evaluate on the validation set
    model.eval()
    val_correct_preds = 0
    val_total_preds = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct_preds += (predicted == labels).sum().item()
            val_total_preds += labels.size(0)
    
    val_accuracy = val_correct_preds / val_total_preds
    print(f"Validation Accuracy with CosineAnnealingLR parameters: {val_accuracy:.4f}")
    
    # Update the best CosineAnnealingLR parameters if the current model performs better
    if val_accuracy > best_cosine_annealing_lr_val_accuracy:
        best_cosine_annealing_lr_warm_val_accuracy = val_accuracy
        best_cosine_annealing_lr_warm_params = scheduler_params

print(f"Best CosineAnnealingLR parameters: {best_cosine_annealing_lr_warm_params}")
print(f"Best CosineAnnealingLR validation accuracy: {best_cosine_annealing_lr_warm_val_accuracy:.4f}")


In [ ]:
# Save the best model
with open('saved_models/CosineAnnealingLR_warm_bestParams.pkl', 'wb') as f:
    pickle.dump(best_cosine_annealing_lr_warm_params, f)

In [ ]:
# Load the saved best model
with open('saved_models/CosineAnnealingLR_warm_bestParams.pkl', 'rb') as f:
    best_cosine_annealing_lr_warm_params = pickle.load(f)

print(f"Loaded Best Hyperparameters: {best_cosine_annealing_lr_warm_params}")

## Model with Best Hyperparameters

#### stepLR

In [ ]:
# Combine train and validation datasets
full_train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
full_train_loader = DataLoader(full_train_dataset, batch_size=512, shuffle=True, num_workers=4, pin_memory=True)

# Initialize the best model with the best hyperparameters
retrained_model_stepLR = DroughtClassifier(
    input_size=input_size,
    hidden_sizes=best_params['hidden_sizes'],
    output_size=output_size,
    dropout_prob=best_params['dropout_prob']
).to(device)


# Define the criterion, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(retrained_model_stepLR.parameters(), lr=best_params['lr'])
scheduler = create_scheduler(optimizer, best_step_lr_params)

print(retrained_model_stepLR)
print(best_params)
print(best_step_lr_params)

In [ ]:
# Retrain the model on the full training dataset
print("Retraining the best model on the full training dataset...")

train_model(retrained_model_stepLR, full_train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=1000, patience=10, log_dir=get_log_dir(), hparams=best_params)

In [ ]:
# Evaluate the retrained model using the test set
evaluate_model(retrained_model_stepLR, test_loader, criterion)

# Save the retrained model
with open('saved_models/retrained_model_stepLR.pkl', 'wb') as f:
    pickle.dump(retrained_model_stepLR, f)

print("Retraining complete. Model saved as 'retrained_model_stepLR.pkl'.")

In [ ]:
# Load the retrained model
with open('saved_models/retrained_model_stepLR.pkl', 'rb') as f:
    retrained_model_stepLR = pickle.load(f)

#### CosineAnnealingLRWarm

In [ ]:
with open('saved_models/CosineAnnealingLR_warm_bestParams.pkl', 'rb') as f:
    best_cosine_annealing_lr_params = pickle.load(f)

with open('saved_models/parameterGridSearch_bestParams.pkl', 'rb') as f:
    best_params = pickle.load(f)


print(best_cosine_annealing_lr_params)
print(best_params)

In [ ]:
# Combine train and validation datasets
full_train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
full_train_loader = DataLoader(full_train_dataset, batch_size=512, shuffle=True, num_workers=4, pin_memory=True)

# Initialize the best model with the best hyperparameters
retrained_model_CosineAnnealingLRWarm = DroughtClassifier(
    input_size=input_size,
    hidden_sizes=best_params['hidden_sizes'],
    output_size=output_size,
    dropout_prob=best_params['dropout_prob']
).to(device)


# Define the criterion, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(retrained_model_CosineAnnealingLRWarm.parameters(), lr=best_params['lr'])
scheduler = create_scheduler(optimizer, best_cosine_annealing_lr_params)

print(retrained_model_CosineAnnealingLRWarm)
print(best_params)
print(best_cosine_annealing_lr_params)

## Model with Best Hyperparameters

In [ ]:
# Retrain the model on the full training dataset
print("Retraining the best model on the full training dataset...")

train_model(retrained_model_CosineAnnealingLRWarm, full_train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=1000, patience=10, log_dir=get_log_dir(), hparams=best_params)

In [ ]:
# Evaluate the retrained model using the test set
evaluate_model(retrained_model_CosineAnnealingLRWarm, test_loader, criterion)

# Save the retrained model
with open('saved_models/retrained_model_CosineAnnealingLRWarm2.pkl', 'wb') as f:
    pickle.dump(retrained_model_CosineAnnealingLRWarm, f)

print("Retraining complete. Model saved as 'retrained_model_CosineAnnealingLRWarm2.pkl'.")

In [ ]:
# Load the retrained model
with open('saved_models/retrained_model_CosineAnnealingLRWarm2.pkl', 'rb') as f:
    retrained_model_CosineAnnealingLRWarm = pickle.load(f)